In [1]:
%load_ext autotime

In [2]:
from sklearn.model_selection import train_test_split
from autoPyTorch import (AutoNetClassification)
import pandas as pd
import numpy as np
import os as os
import json
from sklearn.metrics import accuracy_score

train = pd.read_csv('D:/Data/fashion-mnist/data/archive/fashion-mnist_train.csv')
test = pd.read_csv('D:/Data/fashion-mnist/data/archive/fashion-mnist_test.csv')
X = train.drop(columns=['label'],axis=1)
y = train.label
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17, test_size=0.2)

autoPyTorch = AutoNetClassification("tiny_cs",  # config preset
                                   log_level='debug',
                                    use_tensorboard_logger=True,
                                    #save_models=True !gives error!
                                    cross_validator='k_fold',
                                    cuda=True,
                                    full_eval_each_epoch = True,
                                    validation_split=0.2,
                                    budget_type='epochs',
                                    result_logger_dir='Experiment_logs/Zalando/',
                                    random_seed=17,
                                    max_runtime=300,
                                    min_budget=1,
                                    max_budget=10)

D:\miniconda3\envs\pynew\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
time: 6.7 s
D:\miniconda3\envs\pynew\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
autoPyTorch.fit(X_train, y_train, cross_validator_args={"n_splits": 20}, num_iterations=10, max_runtime=7000, max_budget=200)


:57 Finished train with budget 0.12345679012345678: Preprocessing took 10s, Training took 39s, Wrap up took 0s. Total time consumption in s: 50
22:45:57 [AutoNet] Done with current split!
22:45:57 [AutoNet] CV split 19 of 20
22:45:57 Fit: Imputation
22:46:03 Fit: NormalizationStrategySelector
22:46:05 Fit: OneHotEncoding
22:46:05 Fit: PreprocessorSelector
22:46:09 Fit: ResamplingStrategySelector
22:46:09 Fit: EmbeddingSelector
22:46:09 Fit: NetworkSelector
22:46:09 Fit: InitializationSelector
22:46:09 Fit: OptimizerSelector
22:46:09 Fit: LearningrateSchedulerSelector
22:46:09 Fit: LogFunctionsSelector
22:46:09 Fit: MetricSelector
22:46:09 Fit: LossModuleSelector
22:46:09 Fit: CreateDataLoader
22:46:09 Fit: TrainNode
22:46:09 Start train. Budget: 0.12345679012345678
D:\miniconda3\envs\pynew\lib\site-packages\torch\optim\lr_scheduler.py:509: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)
D:\m

{'optimized_hyperparameter_config': {'CreateDataLoader:batch_size': 125,
  'Imputation:strategy': 'median',
  'InitializationSelector:initialization_method': 'default',
  'InitializationSelector:initializer:initialize_bias': 'No',
  'LearningrateSchedulerSelector:lr_scheduler': 'cosine_annealing',
  'LossModuleSelector:loss_module': 'cross_entropy_weighted',
  'NetworkSelector:network': 'shapedresnet',
  'NormalizationStrategySelector:normalization_strategy': 'standardize',
  'OptimizerSelector:optimizer': 'sgd',
  'PreprocessorSelector:preprocessor': 'truncated_svd',
  'ResamplingStrategySelector:over_sampling_method': 'none',
  'ResamplingStrategySelector:target_size_strategy': 'none',
  'ResamplingStrategySelector:under_sampling_method': 'none',
  'TrainNode:batch_loss_computation_technique': 'standard',
  'LearningrateSchedulerSelector:cosine_annealing:T_max': 10,
  'LearningrateSchedulerSelector:cosine_annealing:eta_min': 2,
  'NetworkSelector:shapedresnet:activation': 'relu',
  '

time: 2h 10min 29s


In [4]:
import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics

y_pred = autoPyTorch.predict(X_test)
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_pred))


Accuracy score 0.8581666666666666
time: 4.18 s


In [6]:
results_refit = autoPyTorch.refit(X,
                              y,
                              autonet_config=autoPyTorch.get_current_autonet_config(),
                              )

23:36:28 Fit: AutoNetSettings
23:36:28 Start autonet with config:
{'embeddings': ['none'], 'lr_scheduler': ['cosine_annealing'], 'networks': ['shapedresnet'], 'preprocessors': ['truncated_svd'], 'target_size_strategies': ['none'], 'over_sampling_methods': ['none'], 'under_sampling_methods': ['none'], 'batch_loss_computation_techniques': ['standard'], 'imputation_strategies': ['median'], 'initialization_methods': ['default'], 'loss_modules': ['cross_entropy_weighted'], 'normalization_strategies': ['standardize'], 'optimizer': ['sgd'], 'hyperparameter_search_space_updates': <autoPyTorch.utils.hyperparameter_search_space_update.HyperparameterSearchSpaceUpdates object at 0x000001A5F7759978>, 'log_level': 'debug', 'use_tensorboard_logger': True, 'cross_validator': 'k_fold', 'cuda': True, 'full_eval_each_epoch': True, 'validation_split': 0.2, 'budget_type': 'epochs', 'result_logger_dir': 'Experiment_logs/Zalando/', 'random_seed': 17, 'max_runtime': 7000, 'min_budget': 1, 'max_budget': 200, '

In [7]:

with open("Experiment_logs/Zalando/results_refit.json", "w") as file:
    json.dump(results_refit, file)

zalando_model = autoPyTorch.get_pytorch_model()
print(zalando_model)    

Sequential(
  (0): Linear(in_features=100, out_features=346, bias=True)
  (1): Sequential(
    (0): ResBlock(
      (layers): Sequential(
        (0): BatchNorm1d(346, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
        (2): Linear(in_features=346, out_features=346, bias=True)
        (3): BatchNorm1d(346, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU()
        (5): Linear(in_features=346, out_features=346, bias=True)
      )
    )
    (1): ResBlock(
      (layers): Sequential(
        (0): BatchNorm1d(346, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): ReLU()
        (2): Linear(in_features=346, out_features=346, bias=True)
        (3): BatchNorm1d(346, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU()
        (5): Linear(in_features=346, out_features=346, bias=True)
      )
    )
    (2): ResBlock(
      (layers): Sequential(
        (0): BatchNo

In [8]:
import pickle

with open('Experiment_logs/Zalando/zalando.pickle', 'wb') as f:
    pickle.dump(zalando_model, f)


time: 1.59 s
